In [ ]:
import numpy as np

In [ ]:
def calculate_iou_mask(mask1, mask2):
    # mask1 and mask2 should be binary numpy arrays of the same shape
    intersection = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    iou = intersection / union
    return iou

def iou_of_mask_sets(set1_masks, set2_masks):
    ious = []
    
    # Ensure the sets are of equal length or handle differently if not
    assert len(set1_masks) == len(set2_masks), "Sets must have the same number of images"
    
    for mask1, mask2 in zip(set1_masks, set2_masks):
        iou = calculate_iou_mask(mask1, mask2)
        ious.append(iou)